In [9]:
pip install chromadb

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 5.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.7/118.7 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 17.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 6.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 4.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 5.7 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 5.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
pip install langchain_openai

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55.0 kB 1.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from operator import itemgetter
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import WebBaseLoader, DirectoryLoader
from langchain.load import dumps, loads
from langchain_core.documents import Document
from openai import OpenAI
import os
import json

In [5]:
bge_model_path = "./bge-small-zh-v1.5/"

client = OpenAI(
    api_key = "sk-f70da689860944fca980b2ee34f3b068",
    base_url = "https://api.deepseek.com",
)

In [6]:
#定义RRF算法函数
def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents 
        and an optional parameter k used in the RRF formula """
    
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}
 
    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)
 
    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]
 
    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

In [19]:

# 0.加载bge embedding模型
bge_embeddings = HuggingFaceBgeEmbeddings(model_name=bge_model_path)
 
# 1.加载文档
loader = DirectoryLoader('./data', glob="**/*.txt")
docs = loader.load()
 
# 2.创建文档分割器，并分割文档
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512,chunk_overlap=0)
splits = text_splitter.split_documents(docs)
 
# 3.创建向量数据库
vectorstore = Chroma.from_documents(documents=splits,embedding=bge_embeddings)
# 4.创建检索器
retriever = vectorstore.as_retriever()
 
 
template1 = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template1)
 
generate_queries = (
    prompt_rag_fusion 
    | ChatOpenAI(model="deepseek-chat", api_key="sk-f70da689860944fca980b2ee34f3b068", base_url="https://api.deepseek.com", temperature=0)
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)
 
 
retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
 
 
# RAG
template2 = """Answer the following question based on this context:
{context}
Question: {question}
"""
 
prompt = ChatPromptTemplate.from_template(template2)
 
final_rag_chain = (
    {"context": retrieval_chain_rag_fusion, 
     "question": itemgetter("question")} 
    | prompt
    | ChatOpenAI(model="deepseek-chat", api_key="sk-f70da689860944fca980b2ee34f3b068", base_url="https://api.deepseek.com", temperature=0)
    | StrOutputParser()
)
 
question='恐龙是怎么灭绝的'
final_rag_chain.invoke({"question":question})

'根据提供的文档内容，恐龙的灭绝主要是由于约0.65亿年前一颗直径约10公里的小行星撞击地球所引发的一系列环境剧变。这一撞击事件发生在现今墨西哥境内，导致了全球范围内的气候和环境变化，最终导致了恐龙的灭绝。\n\n具体来说，小行星撞击地球后，产生了大量的尘埃和碎片，这些物质漂浮在大气中，阻挡了太阳光线，导致地球温度急剧下降，植物生长受到严重影响。随着植物的死亡，食草动物因缺乏食物而大量死亡，进而导致食肉动物也因食物短缺而灭绝。此外，撞击还引发了海啸、地震、野火等自然灾害，进一步加剧了生态系统的崩溃。\n\n尽管还有其他一些理论（如气候变化、火山活动、大气成分变化等）试图解释恐龙的灭绝，但小行星撞击地球的理论得到了较多的科学支持，尤其是通过陨石坑中发现的铱元素异常含量等证据，进一步验证了这一理论的合理性。\n\n因此，恐龙的灭绝主要是由于小行星撞击地球引发的全球性环境剧变所导致的。'